# Установка библиотек

In [320]:
import pandas as pd #для работы с таблицами

Для автоматического применения статистического критерия (теста) используется модуль `stats` библиотеки `scipy`.

Этот модуль содержит большое количество вероятностных распределений, корреляционные функции и статистические тесты и многое другое.

Также активно используется библиотека `statsmodels`

Для установки библиотеки `scipy` и `statsmodels` можно воспользоваться менеджером пакетов `pip`.

In [321]:
!pip install scipy

In [322]:
!pip install statsmodels

-----

В качестве данных возьмем информацию по размерам раковины мидий, выращенных в разных местах.

# Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.


В качестве данных возьмем информацию по размерам раковины мидий, выращенных в разных местах.

In [323]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

Создаем два DataFrame с ключевым столбцом 'num'

In [324]:
data_musseles_petersburg = pd.DataFrame({
    'num': [1,2,3,4,5,6,7],
    'petersburg': [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
})

In [325]:
data_musseles_magadan = pd.DataFrame({
    'num': [1,2,3,4,5,6,7,8], 
    'magadan': [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]
})

Объудиняем таблицы с помощью метода merge по ключевому столбцу num.

In [326]:
data_musseles = data_musseles_magadan.merge(
    data_musseles_petersburg,
    on='num',
    how='left'
)

После объединения удаляем неинформативный столбец num.

In [327]:
data_musseles = data_musseles.drop(['num'], axis=1)

Выводим матрицу корреляции признаков.

In [328]:
data_musseles.corr()

,magadan,petersburg
magadan,1.000000,0.255095
petersburg,0.255095,1.000000


Связь между признаками слабая.

# Проверка данных на нормальность

In [329]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

Установим уровень значимости

In [330]:
alpha = 0.05

Так как данных немного, пользуемся тестом Шапиро-Уилка.

## Тест Шапиро-Уилка

In [331]:
from scipy.stats import shapiro

In [332]:
_, p = shapiro(data_musseles)
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=1.000
Данные распределены нормально


Результат получился слишком хороший, поэтому проверим нормальность еще тестом Д'Агостино.

## Тест Д’Агостино

In [333]:
from scipy.stats import normaltest

In [334]:
_, p = normaltest(data_musseles)
#normaltest возвращает двустороннюю вероятность для проверки гипотезы

print('p=%.3f' % p[0])

# Интерпретация 

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.154
Данные распределены нормально


C:\Users\ksige\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


Наши данные распределены нормально.

Зависимая переменная (размер мидии) является количественной. Группы происходят из разных совокупностей. Следовательно, мы используем независимый T-тест.

# Независимый T-тест 

In [335]:
from scipy.stats import ttest_ind

In [336]:
H0 = 'Нет значимой разницы между размерами мидий в разных городах.'
Ha = 'Есть значимая разница между размерами мидий в разных городах.'

In [337]:
def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(data_musseles['petersburg'], data_musseles['magadan'], equal_var=True)

    p = round(test_results[1],2)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(data_musseles)


*** Результаты независимого T-теста ***
nan <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между размерами мидий в разных городах.


## Выводы: Мы отвергаем нулевую гипотезу. Есть значимая разница между размерами мидий в двух разных городах.